In [13]:
# TESTS THAT INTEGRATION WITH R WORKS
# https://github.com/rpy2/rpy2/issues/1018
# C:\Users\pedro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\rpy2

from rpy2 import situation
import os

try:
    import rpy2.robjects as robjects
except OSError as e:
    try:
        import os
        import platform
        
        r_home = situation.r_home_from_registry()
        r_bin = r_home + '\\bin\\x64\\'
        os.environ['R_HOME'] = r_home
        os.environ['PATH'] =  r_bin + ";" + os.environ['PATH']
        os.add_dll_directory(r_bin)

        print('R_HOME', r_home)
        import rpy2.robjects as robjects
    except OSError:
        raise(e)

print("R Worked!")

R Worked!


In [14]:
import pandas as pd

def compute_theta(filepath):

    # Read the content of the R script file
    r_script_file = "test.R"
    with open(r_script_file, 'r') as file:
        r_script = file.read()

    robjects.r.assign('response_pattern_filepath', f"{filepath}/responses/samples.csv")
    robjects.r.assign('theta_file', f"{filepath}/responses/samples_with_theta.csv")
    robjects.r(r_script)

    robjects.r.assign('response_pattern_filepath', f"{filepath}/responses/random_samples.csv")
    robjects.r.assign('theta_file', f"{filepath}/responses/random_samples_with_theta.csv")
    robjects.r(r_script)

    thetas_df = pd.read_csv(f"{filepath}/responses/samples_with_theta.csv")
    thetas_random_df = pd.read_csv(f"{filepath}/responses/random_samples_with_theta.csv")
    print("thetas.csv finished! rows:", thetas_df.shape[0])

    average_df = thetas_df.groupby('CTT_SCORE')['IRT_SCORE'].mean().reset_index()
    average_random_df = thetas_random_df.groupby('CTT_SCORE')['IRT_SCORE'].mean().reset_index()
    
    return thetas_df, average_df, thetas_random_df, average_random_df

In [15]:
import os
import zipfile
import tempfile
import random
import string

def compute_everything(directory):
    # Get a list of all files in the specified directory
    file_list = os.listdir(directory)

    for file_name in file_list:
        print('file_name', file_name)
        # Check if the file is a zip file
        if file_name.endswith(".zip") and "with_irt" not in file_name:
            zip_path = os.path.join(directory, file_name)

            final_dir = directory + "/" + file_name.split(".zip")[0] + "_with_irt"
            
            # Create a ZipFile object
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                # Extract the contents to the temporary directory
                zip_ref.extractall(final_dir)
                print(f"Files extracted to: {final_dir}")
                
                thetas_df, average_df, thetas_random_df, average_random_df = compute_theta(final_dir)
                average_df.to_csv(final_dir + "/responses/average_theta_by_score_sample.csv")
                average_random_df.to_csv(final_dir + "/responses/average_theta_by_score_random_sample.csv")

# Example usage:
directory_path = "C:/Users/pedro/Downloads/TRI/test_responses_llms/"
compute_everything(directory_path)


file_name exp_CH_mistral_bits_4_count_120.zip
Files extracted to: C:/Users/pedro/Downloads/TRI/test_responses_llms//exp_CH_mistral_bits_4_count_120_with_irt
[1]b' "\x02\xff\xfeRunning...\x03\xff\xfe"'
[1]b' "\x02\xff\xfeLoaded item params\x03\xff\xfe"'
[1]b' "\x02\xff\xfemirt_params.csv Written.\x03\xff\xfe"'
[1]b' "\x02\xff\xfeLoading response patterns...\x03\xff\xfe"'
[1]b' "\x02\xff\xfeLOADED MODEL\x03\xff\xfe"'
[1]b' "\x02\xff\xfeRunning...\x03\xff\xfe"'
[1]b' "\x02\xff\xfeLoaded item params\x03\xff\xfe"'
[1]b' "\x02\xff\xfemirt_params.csv Written.\x03\xff\xfe"'
[1]b' "\x02\xff\xfeLoading response patterns...\x03\xff\xfe"'
[1]b' "\x02\xff\xfeLOADED MODEL\x03\xff\xfe"'
thetas.csv finished! rows: 30000
file_name exp_CH_mistral_bits_4_count_120_with_irt
file_name exp_CH_phi2_bits_4_count_10.zip
Files extracted to: C:/Users/pedro/Downloads/TRI/test_responses_llms//exp_CH_phi2_bits_4_count_10_with_irt
[1]b' "\x02\xff\xfeRunning...\x03\xff\xfe"'
[1]b' "\x02\xff\xfeLoaded item params\x03\